In [30]:
import pandas as pd
import numpy as np
import altair as alt
import streamlit as st

## Wages

Data: https://data.bls.gov/cew/apps/table_maker/v4/table_maker.htm#
如果打不开https://data.bls.gov/cew/apps/data_views/data_views.htm#tab=Tables
在下面18.One area, one industry, quarterly 选2019-2022 All Ownerships，我没找到下载键所以手动输入了loll

<img src='wages.png'>

In [31]:
data = {'time': ['2019Q1', '2019Q2', '2019Q3', '2019Q4',
                '2020Q1', '2020Q2', '2020Q3', '2020Q4',
                '2021Q1', '2021Q2', '2021Q3', '2021Q4',
                '2022Q1', '2022Q2', '2022Q3'],
        'Average Weekly Wage': [1183, 1094, 1092, 1185, 1221, 
                                1188, 1172, 1339, 1288, 1241, 1250, 1418, 1374, 1292, 1334]}
wage = pd.DataFrame(data)
# wage

In [32]:
wage['change'] = round((1183-1145)/1145*100,2) #2018Q4:1145
for i in range(1, len(wage)):
    wage['change'][i] = round((wage['Average Weekly Wage'][i]-wage['Average Weekly Wage'][i-1])\
    /wage['Average Weekly Wage'][i-1]*100,2)
# wage

/var/folders/x8/084zgd0s5r7g56xgsy4qv4w40000gn/T/ipykernel_9276/1331420731.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wage['change'][i] = round((wage['Average Weekly Wage'][i]-wage['Average Weekly Wage'][i-1])\


## General Look on Real Gross Domestic Product and Related Measures: Percent Change from Preceding Period

In [33]:
xls = pd.ExcelFile('gdp4q22_2nd.xlsx')
df1 = pd.read_excel(xls, 'Table 1', header=1)

In [34]:
df = df1.set_index('Unnamed: 1').T
df.reset_index(inplace=True)

In [35]:
df = df.drop(df.columns[[0]],axis = 1)
df = df.drop(df.index[[0,1,2,3]],axis = 0)
df.columns.values[0] = "year"
df.columns.values[1] = "quarter"
df['time'] = df['year'].astype(str)+df['quarter']

In [36]:
consumption = df[['time', 'Gross domestic product (GDP)', 'Personal consumption expenditures',\
                 'Gross private domestic investment', 'Exports', 'Imports',\
                 'Government consumption expenditures and gross investment']]
consumption['time'] = consumption['time'].str[:6]

/var/folders/x8/084zgd0s5r7g56xgsy4qv4w40000gn/T/ipykernel_9276/3803961988.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consumption['time'] = consumption['time'].str[:6]


In [37]:
dat = consumption.merge(wage[['time', 'change']], on='time', how='left')

In [38]:
# dat

In [39]:
dat_long = pd.melt(dat, id_vars='time', value_vars=['Gross domestic product (GDP)', \
                                                    'Personal consumption expenditures', \
                                                   'Gross private domestic investment',
                                                   'Exports', 'Imports', \
                                                    'Government consumption expenditures and gross investment',
                                                   'change'])

In [40]:
variables=list(dat_long['variable'].unique())
variables = variables[:-1]

selectQ=alt.selection_single(
    fields=['variable'],
    init={'variable':variables[0]},
    bind=alt.binding_select(options=variables,name='Select Measures: ')
)

colorCondition = alt.condition(selectQ, 'variable:N', alt.value('lightgray'))
opacityCondition = alt.condition(selectQ, alt.value(1), alt.value(0.4))

base = alt.Chart(dat_long).encode(
    x='time',
).properties(
    height=400,
    width=650
)

q_chart = base.transform_filter(
    alt.datum.variable != 'change'
).mark_line().encode(
    y=alt.Y('value:Q', title='Change on Measures (%)'),
    color='variable:O'
).add_selection(
    selectQ
).encode(
    color=colorCondition,
    opacity=opacityCondition
)

death_chart = base.transform_filter(
    alt.datum.variable == 'change'
).mark_line(color='purple').encode(
    y=alt.Y('value:Q', title='Change on Average Weekly Wage (%)', 
            axis=alt.Axis(titleColor='purple'), scale=alt.Scale(domain=[-80, 100]))
)

gdp=alt.layer(q_chart, death_chart).resolve_scale(
    y = 'independent', color='independent'
).properties(title='Percent Change From Preceding Period on GDP Measures and Average Weekly Wage')
# gdp

### Specific Look on Goods

In [41]:
df2 = pd.read_excel(xls, 'Table 2', header=1)
df = df2.set_index('Unnamed: 1').T
df.reset_index(inplace=True)
df = df.drop(df.columns[[0]],axis = 1)
df = df.drop(df.index[[0,1,2,3]],axis = 0)
df.columns.values[0] = "year"
df.columns.values[1] = "quarter"
df['time'] = df['year'].astype(str)+df['quarter']
df['time'] = df['time'].str[:6]

In [42]:
goods = df[['Durable goods', 'Motor vehicles and parts', 'Furnishings and durable household equipment',
                     'Recreational goods and vehicles',\
                 'Other durable goods', 'Nondurable goods', 'Food and beverages purchased for off-premises consumption', \
                     'Clothing and footwear', 
                'Gasoline and other energy goods', 'Other nondurable goods', 'time']]
goods = goods.merge(wage[['time', 'change']], on='time', how='left')

In [43]:
goods_long = pd.melt(goods, id_vars='time', value_vars=['Durable goods', 'Motor vehicles and parts', 'Furnishings and durable household equipment',
                     'Recreational goods and vehicles',\
                 'Other durable goods', 'Nondurable goods', 'Food and beverages purchased for off-premises consumption', \
                      'Clothing and footwear', 
                'Gasoline and other energy goods', 'Other nondurable goods', 'change'])

In [44]:
goods_long['type'] = 'Durable'
for i in range(len(goods_long)):
    if goods_long['variable'][i] in ('Nondurable goods',
                                     'Food and beverages purchased for off-premises consumption', \
                     'Imports', 'Clothing and footwear', 
                'Gasoline and other energy goods', 'Other nondurable goods'):
        goods_long['type'][i] = 'Nondurable'
    elif goods_long['variable'][i] == 'change':
        goods_long['type'][i] = 'Wage'

In [45]:
durable = goods_long[(goods_long['type'] == 'Durable') | (goods_long['type'] == 'Wage')]
variables=list(durable['variable'].unique())
variables = variables[:-1][1:]

selectQ=alt.selection_single(
    fields=['variable'],
    init={'variable':variables[0]},
    bind=alt.binding_select(options=variables,name='Select durable goods: ')
)

colorCondition = alt.condition(selectQ, 'variable:N', alt.value('lightgray'))


base = alt.Chart(durable).encode(
    x='time',
).properties(
    height=400,
    width=500
)

goods_chart = base.transform_filter(
    alt.datum.variable =='Durable goods'
).mark_line(color='red').encode(
    y=alt.Y('value:Q', 
            scale=alt.Scale(domain=[-8, 16]),
))

death_chart = base.transform_filter(
    alt.datum.variable == 'change'
).mark_line(color='gray').encode(
    y=alt.Y('value:Q', title='Average Weekly Wage (%)',
           axis=alt.Axis(titleColor='gray')),
)

bar_d = base.transform_filter(
    (alt.datum.variable !='Durable goods')
    &(alt.datum.variable !='change')
).mark_bar().encode(
    y=alt.Y('value:Q', title='Durable Goods (%)', axis=alt.Axis(titleColor='red')),
    color='type:O'
).add_selection(selectQ).transform_filter(selectQ)

durable_line = alt.layer(bar_d+goods_chart, death_chart).resolve_scale(
    y = 'independent'
).properties(title='Durable Goods')

In [46]:
nondurable = goods_long[(goods_long['type'] == 'Nondurable') | (goods_long['type'] == 'Wage')]
variables=list(nondurable['variable'].unique())
variables = variables[:-1][1:]

selectQ=alt.selection_single(
    fields=['variable'],
    init={'variable':variables[0]},
    bind=alt.binding_select(options=variables,name='Select nondurable goods: ')
)

base = alt.Chart(nondurable).encode(
    x='time',
).properties(
    height=400,
    width=500
)

goods_chart2 = base.transform_filter(
    alt.datum.variable =='Nondurable goods'
).mark_line(color='red').encode(
    y=alt.Y('value:Q',
           scale=alt.Scale(domain=[-8, 16])
))

death_chart = base.transform_filter(
    alt.datum.variable == 'change'
).mark_line(color='gray').encode(
    y=alt.Y('value:Q',  title='Average Weekly Wage (%)',
           axis=alt.Axis(titleColor='gray'))
)

bar_n = base.transform_filter(
    (alt.datum.variable !='Nondurable goods')
    &(alt.datum.variable !='change')
).mark_bar().encode(
    y=alt.Y('value:Q', title='Nondurable Goods(%)', axis=alt.Axis(titleColor='red')),
    color='type:O'
).add_selection(selectQ).transform_filter(selectQ)

nondurable_line = alt.layer(bar_n+goods_chart2, death_chart).resolve_scale(
    y = 'independent'
).properties(title='Nondurable Goods')

# (durable_line|nondurable_line).properties(
#     title=alt.TitleParams(
#         text="Goods'Contributions to Percent Change in Real GDP",
#         anchor='middle')
# )

### Services

In [47]:
services = df[['Household consumption expenditures (for services)', 'Housing and utilities', 'Health care',
                     'Transportation services',\
                 'Recreation services', 'Food services and accommodations', 'Financial services and insurance', \
                     'Other services', 'time']]
df['time'] = df['time'].str[:6]
services = services.merge(wage[['time', 'change']], on='time', how='left')
services_long = pd.melt(services, id_vars='time', value_vars=['Household consumption expenditures (for services)', \
                                                              'Housing and utilities', 'Health care',
                     'Transportation services',\
                 'Recreation services', 'Food services and accommodations', 'Financial services and insurance', \
                     'Other services', 'change'])

In [48]:
variables=list(services_long['variable'].unique())
variables = variables[:-1][1:]

selectQ=alt.selection_single(
    fields=['variable'],
    init={'variable':variables[0]},
    bind=alt.binding_select(options=variables,name='Select services: ')
)


base = alt.Chart(services_long).encode(
    x='time',
).properties(
    height=400,
    width=500
)

service_chart = base.transform_filter(
    alt.datum.variable =='Household consumption expenditures (for services)'
).mark_line(color='red').encode(
    y=alt.Y('value:Q'),
)

death_chart = base.transform_filter(
    alt.datum.variable == 'change'
).mark_line(color='gray').encode(
    y=alt.Y('value:Q', title='Average Weekly Wage (%)',
           axis=alt.Axis(titleColor='gray'), scale=alt.Scale(domain=[-25, 20]))
)

bar_s = base.transform_filter(
    (alt.datum.variable !='Household consumption expenditures (for services)')
    &(alt.datum.variable !='change')
).mark_bar().encode(
    y=alt.Y('value:Q', title='Household consumption expenditures (for services)(%)',
           axis=alt.Axis(titleColor='red')),
).add_selection(selectQ).transform_filter(selectQ)

service=alt.layer(bar_s+service_chart, death_chart).resolve_scale(
    y = 'independent'
).properties(title="Services' Contributions to Percent Change in Real GDP")
# service

### With Wages

In [49]:
# service|gdp

In [50]:
# (durable_line|nondurable_line).properties(
#     title=alt.TitleParams(
#         text="Goods'Contributions to Percent Change in Real GDP",
#         anchor='middle')
# )

## Without Wages

In [51]:
durable = goods_long[(goods_long['type'] == 'Durable') | (goods_long['type'] == 'Wage')]
variables=list(durable['variable'].unique())
variables = variables[:-1][1:]

selectQ=alt.selection_single(
    fields=['variable'],
    init={'variable':variables[0]},
    bind=alt.binding_select(options=variables,name='Select durable goods: ')
)

colorCondition = alt.condition(selectQ, 'variable:N', alt.value('lightgray'))


base = alt.Chart(durable).encode(
    x='time',
).properties(
    height=400,
    width=500
)

goods_chart = base.transform_filter(
    alt.datum.variable =='Durable goods'
).mark_line(color='red').encode(
    y=alt.Y('value:Q', 
#             scale=alt.Scale(domain=[-8, 16])
))

bar_d = base.transform_filter(
    (alt.datum.variable !='Durable goods')
    &(alt.datum.variable !='change')
).mark_bar().encode(
    y=alt.Y('value:Q', title='Durable Goods (%)'),
    color='type:O'
).add_selection(selectQ).transform_filter(selectQ)

d = (bar_d+goods_chart).properties(title='Durable Goods')

In [52]:
nondurable = goods_long[(goods_long['type'] == 'Nondurable') | (goods_long['type'] == 'Wage')]
variables=list(nondurable['variable'].unique())
variables = variables[:-1][1:]
nondurable['line_label'] = len(nondurable) * ['overall contribution']

selectQ=alt.selection_single(
    fields=['variable'],
    init={'variable':variables[0]},
    bind=alt.binding_select(options=variables,name='Select nondurable goods: ')
)

base = alt.Chart(nondurable).encode(
    x='time',
).properties(
    height=400,
    width=500
)

goods_chart2 = base.transform_filter(
    alt.datum.variable =='Nondurable goods'
).mark_line(color='red').encode(
    y=alt.Y('value:Q',
#            scale=alt.Scale(domain=[-8, 16])
),
opacity='line_label')

bar_n = base.transform_filter(
    (alt.datum.variable !='Nondurable goods')
    &(alt.datum.variable !='change')
).mark_bar().encode(
    y=alt.Y('value:Q', title='Nondurable Goods(%)'),
    color='type:O'
).add_selection(selectQ).transform_filter(selectQ)

nd = (bar_n+goods_chart2).properties(title='Nondurable Goods')

/var/folders/x8/084zgd0s5r7g56xgsy4qv4w40000gn/T/ipykernel_9276/3720421527.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nondurable['line_label'] = len(nondurable) * ['overall contribution']


In [53]:
goods = (d|nd).properties(
    title=alt.TitleParams(
        text="Goods' Contributions to Percent Change in Real GDP",
        anchor='middle')
)

In [54]:
variables=list(services_long['variable'].unique())
variables = variables[:-1][1:]
services_long['line_label'] = len(services_long) * ['overall contribution']

selectQ=alt.selection_single(
    fields=['variable'],
    init={'variable':variables[0]},
    bind=alt.binding_select(options=variables,name='Select services: ')
)


base = alt.Chart(services_long).encode(
    x='time',
).properties(
    height=400,
    width=500
)

service_chart = base.transform_filter(
    alt.datum.variable =='Household consumption expenditures (for services)'
).mark_line(color='red').encode(
    y=alt.Y('value:Q', scale=alt.Scale(domain=[-25,25])),
    opacity='line_label',
).properties(title="Services' Contributions to Percent Change in Real GDP")

bar_s = base.transform_filter(
    (alt.datum.variable !='Household consumption expenditures (for services)')
    &(alt.datum.variable !='change')
).mark_bar().encode(
    y=alt.Y('value:Q', title='Household consumption expenditures (for services)(%)'),
).add_selection(selectQ).transform_filter(selectQ)

service2 = (bar_s+service_chart)

In [55]:
st.title("Percent Change of GDP measures and Average Weekly Wage Quarterly Since 2019 to 2022")
st.write("The visualization below shows the percent change of real GDP versus the average weekly wage.")
st.write("Additionally, the users could select GDP's related measure through the dropdown box to make comparisons with the change of wage.")

In [56]:
gdp

alt.LayerChart(...)

In [57]:
# goods
st.write("To further investigate the influence on personal consumption expenditures, we visualize the contributions of goods and services to percent change in real GDP.")
st.write("The line in each plot shows the overall contributions, while the users could look at the specific contribution of a particular good or service through the bar plot by using the dropdown selection.")
goods

alt.HConcatChart(...)

In [58]:
service2

alt.LayerChart(...)